In [37]:
import sagemaker
import boto3
import pandas as pd

from sagemaker import image_uris
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput
from time import gmtime, strftime
import time
import sagemaker.amazon.common as smac
import numpy as np
import io
import os

region = boto3.Session().region_name

session = sagemaker.Session()
bucket = 'lawsnic-aiml-east2'
prefix = 'kaggle/customerChurn'

role = sagemaker.get_execution_role()

sm = boto3.Session().client(service_name='sagemaker',region_name=region)

In [38]:
# cell 10
container = sagemaker.image_uris.retrieve(region=boto3.Session().region_name, framework='xgboost', version='latest')

In [39]:
# cell 11
training_location = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/test/CustomerChurnDW-2023-01-06T17-24-14/part-00000-3986a971-e8a5-4844-bfc8-e72e78f341b4-c000.csv'
validation_location = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/train/CustomerChurnDW-2023-01-06T17-24-14/part-00000-0338ba51-05d0-4737-bd40-d52e24739eb6-c000.csv'

df = pd.read_csv(training_location)
#df.to_csv('s3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/train.csv',header=None,index=False)

#df = pd.read_csv(validation_location)
#df.to_csv('s3://fsb-develop-cb-ml-input-snowflake/attrite_target.csv',header=None,index=False)


In [40]:
train_data = df.to_numpy()
# split into train/test with a 90/10 split
np.random.seed(0)
np.random.shuffle(train_data)

n_train_size = int(0.9 * train_data.shape[0])
n_train_features = train_data[:n_train_size, :-1]

n_train_features = train_data[:n_train_size, :-1]
n_train_labels = train_data[:n_train_size, -1]
n_test_features = train_data[n_train_size:, :-1]
n_test_labels = train_data[n_train_size:, -1]


# write to buffer
buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, n_train_features, n_train_labels)
buf.seek(0)






boto3.resource('s3').Bucket('lawsnic-aiml-east2').Object(os.path.join('recordIOProto', 'train_SGPCA_recordIOProto')).upload_fileobj(buf)

s3_train_data = 's3://lawsnic-aiml-east2/recordIOProto/train_SGPCA_recordIOProto'

First we'll need to specify training parameters to the estimator.  This includes:
1. The `xgboost` algorithm container
1. The IAM role to use
1. Training instance type and count
1. S3 location for output data
1. Algorithm hyperparameters

And then a `.fit()` function which specifies:
1. S3 location for output data.  In this case we have both a training and validation set which are passed in.

In [ ]:
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/manual-output/xgb-hpo'.format(bucket,prefix),
                                    sagemaker_session=sess,
                                    enable_sagemaker_metrics=True)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        eval_metric='auc',
                        num_round=100)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation}) 

In [14]:
training_job_name = f"DEMO-xgboost-regression-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"

# Ensure that the training and validation data folders generated above are reflected in the "InputDataConfig" parameter below.

create_training_params = {
    "AlgorithmSpecification": {"TrainingImage": container, "TrainingInputMode": "File"},
    "RoleArn": role,
    "OutputDataConfig": {"S3OutputPath": f"s3://{bucket}/{prefix}/manual-output/xgboost-creatjob"},
    "ResourceConfig": {"InstanceCount": 1, "InstanceType": "ml.m4.xlarge", "VolumeSizeInGB": 5},
    "TrainingJobName": training_job_name,
    "HyperParameters": {
        "max_depth": "5",
        "eta": "0.2",
        "gamma": "4",
        "min_child_weight": "6",
        "subsample": "0.7",
        "objective": "binary:logistic",
        "num_round": "50",
        "verbosity": "2",
        "eval_metric":"auc"
    },
    "StoppingCondition": {"MaxRuntimeInSeconds": 3600},
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": f"s3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/train.csv",
                    "S3DataDistributionType": "FullyReplicated",
                }
            },
            "ContentType": "csv",
            "CompressionType": "None",
        },
        {
            "ChannelName": "validation",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": f"s3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/validate.csv",
                    "S3DataDistributionType": "FullyReplicated",
                }
            },
            "ContentType": "csv",
            "CompressionType": "None",
        },
    ],
}

print(f"Creating a training job with name: {training_job_name}. It will take between 5 and 6 minutes to complete.")
sm.create_training_job(**create_training_params)
status = sm.describe_training_job(TrainingJobName=training_job_name)["TrainingJobStatus"]

print(status)

#while status != "Completed" and status != "Failed":
#    time.sleep(60)
#    status = sm.describe_training_job(TrainingJobName=training_job_name)["TrainingJobStatus"]
#    print(status)

Creating a training job with name: DEMO-xgboost-regression-2023-01-08-19-39-10. It will take between 5 and 6 minutes to complete.
InProgress


In [ ]:
boto3.client('sagemaker').describe_training_job(TrainingJobName=tuner.best_training_job())

## HPO

In [29]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
hyperparameter_ranges = {'eta': ContinuousParameter(0, 1),
                            'min_child_weight': ContinuousParameter(1, 10),
                            'alpha': ContinuousParameter(0, 2),
                            'max_depth': IntegerParameter(1, 10)}

objective_metric_name = 'validation:auc'

tuner = HyperparameterTuner(xgb,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=20,
                            max_parallel_jobs=3)

In [30]:
tuner.fit({'train': s3_input_train, 'validation': s3_input_validation})

......................................................................................................!


In [34]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

'Completed'

In [37]:
boto3.client('sagemaker').describe_training_job(TrainingJobName=tuner.best_training_job())['FinalMetricDataList']

[{'MetricName': 'validation:auc',
  'Value': 0.8405889868736267,
  'Timestamp': datetime.datetime(2023, 1, 6, 23, 14, 55, tzinfo=tzlocal())},
 {'MetricName': 'train:auc',
  'Value': 0.8764730095863342,
  'Timestamp': datetime.datetime(2023, 1, 6, 23, 14, 55, tzinfo=tzlocal())},
 {'MetricName': 'ObjectiveMetric',
  'Value': 0.8405889868736267,
  'Timestamp': datetime.datetime(2023, 1, 6, 23, 14, 55, tzinfo=tzlocal())}]